# Gathering fMRI Data
Both anatomical and functional

In [46]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

In [47]:
proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [48]:
raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

# Parse Stimulus Data
I am somewhat mystified by the `csv` formatting for from PsychoPy. I am going to start by parsing it from the log file. This is perhaps not the most robust way to do it, but it will be quicker than reading through and testing all the pyschopy documentation. It also provides a ton of metadata.

In [69]:
import ast
from collections import OrderedDict

In [111]:
def parse_psychopy_log(log_path):
    with open(log_path) as log:
        log = log.readlines()
    for entry in log:
        (timestamp, level, description) = entry.split(" 	")
        if description.split()[1] == "image" and description.split()[0][:6] == "image_":
            #print(description.split(" ")[0][:5])
            #print(description.split()[0][:6])
            # a lil slow but it's readable
            image = description.split()[3].replace("'","")
            print(timestamp, image)


#TODO Frame Info:
# what is on the screen (image)
# What input is happening at that frame (keypresses)
# later: the timeslice of the fMRI
# Integrate into view fMRI
parse_psychopy_log(log_path)

181.9064 POFA/fMRI_POFA/021c.tif
186.8437 POFA/fMRI_POFA/072c.tif
191.8438 POFA/fMRI_POFA/013c.tif
196.8440 POFA/fMRI_POFA/033c.tif
201.8439 POFA/fMRI_POFA/083c.tif
206.8440 POFA/fMRI_POFA/006c.tif
241.8758 POFA/fMRI_POFA/036a.tif
246.8443 POFA/fMRI_POFA/008a.tif
251.8445 POFA/fMRI_POFA/075a.tif
256.8601 POFA/fMRI_POFA/049a.tif
261.8443 POFA/fMRI_POFA/023a.tif
266.8448 POFA/fMRI_POFA/086a.tif
271.8603 POFA/fMRI_POFA/069a.tif
276.8136 POFA/fMRI_POFA/080a.tif
283.4543 POFA/fMRI_POFA/018a.tif
287.1106 POFA/fMRI_POFA/061a.tif
291.7982 POFA/fMRI_POFA/053a.tif
296.8455 POFA/fMRI_POFA/105a.tif
301.8296 POFA/fMRI_POFA/083a.tif
306.8137 POFA/fMRI_POFA/006a.tif
311.8294 POFA/fMRI_POFA/013a.tif
316.8140 POFA/fMRI_POFA/033a.tif
321.8008 POFA/fMRI_POFA/021a.tif
326.7985 POFA/fMRI_POFA/072a.tif
361.7673 POFA/fMRI_POFA/086a.tif
366.7521 POFA/fMRI_POFA/075a.tif
371.7521 POFA/fMRI_POFA/049a.tif
376.7520 POFA/fMRI_POFA/008a.tif
381.7522 POFA/fMRI_POFA/023a.tif
386.7523 POFA/fMRI_POFA/036a.tif
391.7677 P

Okay, this is cool and all, but let's get a working volume in blender before anything else

# Visualize BOLD

According to the [study](https://openneuro.org/datasets/ds003548/versions/1.0.1) the resting state fMRI is ten minutes long. Some chatbots suggested that the 4th index in the `ANTsImage` `Spacing` Tuple would be the time spacing, and correspond to the number of seconds each frame is taken at.

To verify this, let's make sure that $\frac{Slice Duration * Number Of Slices}{60}=10$

Confusingly enough, Dimensions is the name in the return string when printed, while these values are accessed by `.shape`.

In [32]:
print(raw_rest_bold_img)
minutes = (raw_rest_bold_img.spacing[3] * float(raw_rest_bold_img.shape[3])) / 60.0
print(type(raw_rest_bold_img.shape[3]))
print(minutes)

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (64, 64, 35, 300)
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]

<class 'int'>
10.0


Looks good to me. Let's take this assumption that `.spacing[3]` will be the duration of the slices in seconds

In [33]:
with open(events_tsv_path, 'r') as f:
    events_tsv = f.read()

In [34]:
# TODO display actual stimulus
# TODO show all three spatial axes in one display
# TODO rotate these displays
# TODO once integrated add masking

def explore_fMRI(ants_img: ants.core.ants_image.ANTsImage,
                 dim="x", events_tsv="NULL",
                 cmap='nipy_spectral'):
    vol = ants_img.numpy()
    def dim_to_indexed(dim, slice, frame):
        match dim:
            case "x":
                return vol[slice,:,:,frame]
            case "y":
                return vol[:,slice,:,frame]
            case "z":
                return vol[:,:,slice,frame]

    def plot(slice, frame):
        second = float(frame * ants_img.spacing[3])
        plt.figure()
        plt.imshow(dim_to_indexed(dim, slice, frame), cmap=cmap)
        plt.show()
        present_event = "No event file"
        if events_tsv != "NULL":
            for event in events_tsv.split("\n"):
                info = event.split("	")
                if info[0].isdigit() and info[1].isdigit():
                    if float(info[0]) <= second < float(info[0] + info[1]):
                        present_event = info[2]
            print(present_event)

    frame_slider = (0, (vol.shape[3]-1))
    match dim:
        case "x":
            interact(plot, slice=(0, vol.shape[0]-1), frame=frame_slider)
        case "y":
            interact(plot, slice=(0, vol.shape[1]-1), frame=frame_slider)
        case "z":
            interact(plot, slice=(0, vol.shape[2]-1), frame=frame_slider)

In [35]:
explore_fMRI(raw_stim_bold, dim="x", events_tsv=events_tsv)
explore_fMRI(raw_stim_bold, dim="y", events_tsv=events_tsv)
explore_fMRI(raw_stim_bold, dim="z", events_tsv=events_tsv)

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='frame…

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='frame…

interactive(children=(IntSlider(value=17, description='slice', max=34), IntSlider(value=92, description='frame…

# Subtract Baseline from Stimulus to Isolate the Activations

So it looks like this experiment doesn't have a neutral stimulus, just a "rest" . This isn't great, I'd much prefer a Block Design. There are a couple of ways we could build an experiment. One thought I had would be to take the mean of the rest state and then read the activations as being against that? *Is this valid?*

In [41]:
temporal_average_rest = np.mean(raw_rest_bold_img.numpy(), axis=3)

In [42]:
explore_3D_vol(temporal_average_rest)
print(temporal_average_rest.shape)

interactive(children=(IntSlider(value=31, description='slice', max=63), Output()), _dom_classes=('widget-inter…

(64, 64, 35)


In [43]:
def subtract_neutral_3D(experimental, neutral):
    """
    Subtracts a 3D neutral from each time slice in a 4D experimental volume
    For use in experiments which lack a block design and for which the 
    neutral stimulus has been derived from an averaged rest state
    """
    result = np.empty_like(experimental)
    for time_slice in range(experimental.shape[3]):
        result[:,:,:,time_slice] = experimental[:,:,:, time_slice] - neutral
    return result

In [44]:
isolated_BOLD = subtract_neutral_3D(raw_stim_bold.numpy(), temporal_average_rest)

In [45]:
explore_fMRI(isolated_BOLD, dim="y")

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

# Measure Change in Volume
Here's another possible approach. Again, I do not know the scientific validity of this. Here we'll take each frame and measure the difference in the bold response from the previous frame.

Grabbing the absolute value between the current and previous timestamp creates the most convenient isolation for our visualization tool. However, this does not mean that it is the most scientifically valuable.

In [ ]:
def measure_BOLD_movement(bold_vol, baseline_vol):
    """
    Each frame shows the difference between it and the previous frame. First frame is initialized at zero. 
    """
    result = np.empty_like(bold_vol)
    for time_slice in range(1, bold_vol.shape[3]):
        result[:,:,:,time_slice] = np.absolute(bold_vol[:,:,:,time_slice] - bold_vol[:,:,:,time_slice - 1])
    return result

In [ ]:
bold_movement = measure_BOLD_movement(raw_stim_bold.numpy(), temporal_average_rest)

In [ ]:
explore_fMRI(bold_movement)

# Register and Mask BOLD and Anat

We're going to go wit the `bold_movement` volume. Not sure if it's the best, but it's the most convenient to visualize in this context. Or maybe we mask/register the bold first? Is that the best way of going about it?

We're also going to need to account for motion correction and size differences between anat and bold. Oh boy

In [ ]:
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [ ]:
#Slicing methodology check
sliced_bold = ants.from_numpy(raw_stim_bold.numpy()[:,:,:,50])
sliced_bold02 = ants.from_numpy(raw_stim_bold.numpy()[:,:,:,100])
explore_3D_vol(sliced_bold.numpy())
print(sliced_bold is sliced_bold02)

In [ ]:
def bold_masking(bold_img, template, mask, dilate=True):
    indent = "        ➡️"
    print("Masking bold. This might take a while...")
    result = np.empty_like(bold_img.numpy())


    for time_slice in range(bold_img.numpy().shape[3]):
        bold_slice = ants.from_numpy(bold_img.numpy()[:,:,:,time_slice])

        print(f"Masking for time slice {time_slice} out of {bold_img.numpy().shape[3]}")
        print(f"{indent}creating template")
        template_warp_to_bold_anat = ants.registration(
            fixed=bold_slice,
            moving=template, 
            type_of_transform='SyN',
            verbose=False
            )
        
        print(f"{indent}Registering template image")

        print(f"{indent}Creating brain mask")
        brain_mask = ants.apply_transforms(
            fixed=template_warp_to_bold_anat['warpedmovout'],
            moving=mask,
            transformlist=template_warp_to_bold_anat['fwdtransforms'],
            interpolator='nearestNeighbor',
            verbose=False
            )
        if dilate:
            print(f"{indent}Dilating brian mask")
            brain_mask = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')
        print(f"{indent}Applying brain mask and adding to final result")
        result[:,:,:,time_slice] = ants.mask_image(bold_slice, brain_mask).numpy()
    return ants.from_numpy(result)


In [ ]:
#masked_bold = bold_masking(raw_stim_bold, mni_template, mni_mask)

In [ ]:
# explore_4D_vol(masked_bold)
# explore_4D_vol(raw_stim_bold.numpy())

Well, it looks like this is the exact same thing

looking at the mri, though, do we even need to mask the brain? Can we just threshold out the purple stuff?

The following is a very lovely function but it wasn't very smart of you to write it. If you look at the BOLD respoonse in this dataset you can see that we can isolate the brain just with grid-specific threshholding.

# Register BOLD to T1

Like the above function, this will register the BOLD function to the anat. A more typical analysis pipeline might register to MNI space, as above, but we don't really care about voxel-specific cross-study analysis; we want a subject-anatomy specific visualization.

We're reusing much of the logic above, and will have to write custom viewer functions to verify our work

In [ ]:
new_dataset = ants.image_read("/Users/joachimpfefferkorn/Downloads/sub-01122021301_task-arousal_bold.nii.gz")

In [ ]:
explore_fMRI(new_dataset.numpy())

In [ ]:
print(new_dataset)
print(raw_stim_bold)